In [1]:
!ls ../input/

hetero-link-prediction-openi-all  openidataset
openi-link-prediction		  openigraphdata


In [2]:
!pip install torchxrayvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 26.2 MB/s eta 0:00:00


In [3]:
DEBUG = False

In [4]:
!pip install rouge-score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=70e95ab35ae1012b3be27fde8fc8b6eabd559ef8370cbca614384e00a6651889
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score


In [5]:
!pip install -U nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.3 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.7 which is incompatible.


In [6]:
import os
import numpy as np
import pandas as pd
import cv2
import random
import math
import pickle
import nltk
import skimage.io


import torch
from torchvision import transforms, models
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel, VisualBertModel, VisualBertConfig
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import AutoTokenizer, AutoModel
from nltk.translate.bleu_score import sentence_bleu
from tqdm.notebook import tqdm
from PIL import Image
from random import choice
from sklearn.model_selection import GroupKFold
import torchxrayvision as xrv
# from rouge_score import rouge_scorer

from torch.utils.data import random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)


from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate import meteor_score

import nltk
nltk.download('omw-1.4')


import matplotlib.pyplot as plt


[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [7]:
DATA_DIR = '../input/openidataset/images/images/'
DEBUG = False
config = dict(
    seed = 42,
    epochs = 1 if DEBUG else 20,
    max_position_embeddings = 200,
    encoder_max_length = 43,
    decoder_max_length = 232,
    bs = 4 if DEBUG else 4,
    lr = 1e-5,
    patience = 10,
    factor = 0.1,
    image_size = 224,
    hidden_size = 2048,
    hidden_dropout_prob = 0.3,
    vocab_size = 30522
)

In [8]:
def preprocess_text(df,tokenizer,isTrain=True):
#     df['Findings'] = df['Findings'].apply(lambda x : str(x).strip()) 
#     df['Impression'] = df['Impression'].apply(lambda x : str(x).strip())
    
#     findings = df['Findings'].values
#     impression = df['Impression'].values
    
#     new_target = []
#     for fnd,imp in zip(findings,impression):
#         if(fnd == 'none' and imp == 'none'):
#             new_target.append('none')
#         elif(fnd == 'none' and imp != 'none'):
#             new_target.append(imp)
#         elif(fnd != 'none' and imp == 'none'):
#             new_target.append(fnd)
#         else:
#             new_target.append(fnd + ' ' + imp)
            
#     df['Target_New'] = new_target
#     df['Target_New'] = df['Target_New'].apply(lambda x : str(x).strip())

    df['Target'] = df['Target'].apply(lambda x : str(x).strip())
    df['target_tokens'] = df['Target'].apply(lambda x : tokenizer.tokenize(x))
    
#     if(isTrain):
#         df['Keywords'] = df['Keywords'].apply(lambda x : str(x).replace(',',' '))
#         df['Keywords'] = df['Keywords'].apply(lambda x : x.strip())
#         df['keyword_tokens'] = df['Keywords'].apply(lambda x : tokenizer.tokenize(x))
#     else:
    df['Prepoc_Problems'] = df['Prepoc_Problems'].apply(lambda x : str(x).replace(';',' '))
    df['Prepoc_Problems'] = df['Prepoc_Problems'].apply(lambda x : x.strip())
    df['keyword_tokens'] = df['Prepoc_Problems'].apply(lambda x : tokenizer.tokenize(x))
        
    
    # max sequence length after cleaning
    df['target_seq_len'] = df['target_tokens'].apply(lambda x : len(x))
    df['keyword_seq_len'] = df['keyword_tokens'].apply(lambda x : len(x))
    max_seq_len_target = df['target_seq_len'].max()
    max_seq_len_keyword = df['keyword_seq_len'].max()
    print("Max Target Tokens = ", max_seq_len_target)
    print("Max Keyword Tokens = ", max_seq_len_keyword)
    return df,max_seq_len_target,max_seq_len_keyword

In [9]:
img_dir = '../input/openigraphdata/Mesh_Keywords/openi_image_embed/'
word_dir = '../input/openigraphdata/Mesh_Keywords/openi_word_embed/'

In [10]:
with open(img_dir + 'openi_train_img_ids','rb') as f:
    train_img_ids = pickle.load(f)
    
with open(img_dir + 'openi_val_img_ids','rb') as f:
    val_img_ids = pickle.load(f)
    
with open(img_dir + 'openi_test_img_ids','rb') as f:
    test_img_ids = pickle.load(f)

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
train_df = pd.read_csv(word_dir + 'train_reports_with_keywords.csv')
train_df,_,_ = preprocess_text(train_df,tokenizer)
train_df.head()

Max Target Tokens =  232
Max Keyword Tokens =  43


,Image,Frontal,Lateral,Comparison,Indication,Findings,Impression,MESH,Problems,Target,Problem_List,Prepoc_Problems,target_tokens,keyword_tokens,target_seq_len,keyword_seq_len
0,CXR3403_IM-1647-1001;CXR3403_IM-1647-2001,CXR3403_IM-1647-1001,CXR3403_IM-1647-2001,None.,XXXX-year-old female. Left chest pain. Right r...,the cardiomediastinal silhouette is normal in ...,negative for acute abnormality,normal,normal,the cardiomediastinal silhouette is normal in ...,['normal'],normal,"[the, card, ##iom, ##ed, ##ias, ##tina, ##l, s...",[normal],43,1
1,CXR810_IM-2343-1001;CXR810_IM-2343-2001,CXR810_IM-2343-1001,CXR810_IM-2343-2001,None.,chest pain,normal heart size mediastinal and aortic conto...,no evidence of active cardiopulmonary disease,Calcified Granuloma/scattered;Spine/degenerative,Calcified Granuloma;Spine,normal heart size mediastinal and aortic conto...,"['calcified granuloma', 'spine']",calcified granuloma spine,"[normal, heart, size, media, ##sti, ##nal, and...","[cal, ##ci, ##fied, gran, ##ulo, ##ma, spine]",63,7
2,CXR2474_IM-1003-1001,CXR2474_IM-1003-1001,NaN,NaN,Shortness of breath.,patchy interstitial infiltrates have developed...,bibasilar patchy airspace disease with bilater...,Infiltrate/lung/lower lobe/bilateral/interstit...,Infiltrate;Costophrenic Angle;Aorta;Airspace D...,patchy interstitial infiltrates have developed...,"['infiltrate', 'costophrenic angle', 'aorta', ...",infiltrate costophrenic angle aorta airspace d...,"[patch, ##y, inter, ##sti, ##tial, infiltrate,...","[infiltrate, cost, ##op, ##hre, ##nic, angle, ...",53,16
3,CXR632_IM-2213-1001;CXR632_IM-2213-1002,CXR632_IM-2213-1001,CXR632_IM-2213-1002,Chest x-XXXX and XXXX.,"XXXX-year-old male with shortness of breath, X...",cardiac and mediastinal silhouette are unremar...,no acute cardiopulmonary abnormality,normal,normal,cardiac and mediastinal silhouette are unremar...,['normal'],normal,"[cardiac, and, media, ##sti, ##nal, silhouette...",[normal],49,1
4,CXR3101_IM-1453-1001;CXR3101_IM-1453-3001,CXR3101_IM-1453-1001,CXR3101_IM-1453-3001,None.,Multiple myeloma. Bone marrow transplant.,the heart size and pulmonary vascularity appea...,no evidence of active disease,"Spine/degenerative;Stents/abdomen;Aorta, Thora...","Spine;Stents;Aorta, Thoracic;Calcified Granuloma",the heart size and pulmonary vascularity appea...,"['spine', 'stents', 'aorta thoracic', 'calcifi...",spine stents aorta thoracic calcified granuloma,"[the, heart, size, and, pulmonary, vascular, #...","[spine, ste, ##nts, ao, ##rta, thor, ##ac, ##i...",74,14


In [13]:
val_df = pd.read_csv(word_dir + 'val_reports_with_keywords.csv')
val_df,_,_ = preprocess_text(val_df,tokenizer,isTrain=False)
val_df.head()

Max Target Tokens =  218
Max Keyword Tokens =  43


,Image,Frontal,Lateral,Comparison,Indication,Findings,Impression,MESH,Problems,Target,Problem_List,Prepoc_Problems,target_tokens,keyword_tokens,target_seq_len,keyword_seq_len
0,CXR3518_IM-1717-1001;CXR3518_IM-1717-2001,CXR3518_IM-1717-1001,CXR3518_IM-1717-2001,None available.,"XXXX year old with prostate cancer, no chest c...",the heart and mediastinal silhouettes are with...,no acute visualized cardiopulmonary abnormality,Osteophyte/thoracic vertebrae/degenerative/mul...,Osteophyte,the heart and mediastinal silhouettes are with...,['osteophyte'],osteophyte,"[the, heart, and, media, ##sti, ##nal, silhoue...","[os, ##te, ##op, ##hy, ##te]",60,5
1,CXR2553_IM-1059-1001;CXR2553_IM-1059-2001,CXR2553_IM-1059-1001,CXR2553_IM-1059-2001,None,"XXXX-year-old male with XXXX, 786.2",there are several small calcified granulomas t...,no evidence of active disease,Calcified Granuloma/lung/multiple/small;Spine/...,Calcified Granuloma;Spine,there are several small calcified granulomas t...,"['calcified granuloma', 'spine']",calcified granuloma spine,"[there, are, several, small, cal, ##ci, ##fied...","[cal, ##ci, ##fied, gran, ##ulo, ##ma, spine]",71,7
2,CXR3156_IM-1486-1001;CXR3156_IM-1486-2001,CXR3156_IM-1486-1001,CXR3156_IM-1486-2001,None.,XXXX-year-old woman with positive PPD..,the lungs are clear bilaterally specifically n...,no acute cardiopulmonary abnormality specifica...,normal,normal,the lungs are clear bilaterally specifically n...,['normal'],normal,"[the, lungs, are, clear, bilateral, ##ly, spec...",[normal],70,1
3,CXR3883_IM-1971-1001;CXR3883_IM-1971-12012,CXR3883_IM-1971-1001,CXR3883_IM-1971-12012,None.,The patient is a XXXX-year-old female with lef...,no pneumothorax pleural effusion or airspace c...,no acute cardiopulmonary abnormality,normal,normal,no pneumothorax pleural effusion or airspace c...,['normal'],normal,"[no, p, ##ne, ##um, ##otho, ##ra, ##x, pl, ##e...",[normal],40,1
4,CXR330_IM-1577-0001-0001;CXR330_IM-1577-0001-0002,CXR330_IM-1577-0001-0002,CXR330_IM-1577-0001-0001,None available.,XXXX-year-old woman with hepatic encephalopath...,heart size mediastinal contour and pulmonary v...,no acute cardiopulmonary abnormality,normal,normal,heart size mediastinal contour and pulmonary v...,['normal'],normal,"[heart, size, media, ##sti, ##nal, con, ##tour...",[normal],57,1


In [14]:
test_df = pd.read_csv(word_dir + 'test_reports_with_keywords.csv')
test_df,_,_ = preprocess_text(test_df,tokenizer,isTrain=False)
test_df.head()

Max Target Tokens =  280
Max Keyword Tokens =  34


,Image,Frontal,Lateral,Comparison,Indication,Findings,Impression,MESH,Problems,Target,Problem_List,Prepoc_Problems,target_tokens,keyword_tokens,target_seq_len,keyword_seq_len
0,CXR1852_IM-0554-1001;CXR1852_IM-0554-2001,CXR1852_IM-0554-1001,CXR1852_IM-0554-2001,None,Chest pain,lungs are clear no focal consolidation effusio...,no acute cardiopulmonary disease,normal,normal,lungs are clear no focal consolidation effusio...,['normal'],normal,"[lungs, are, clear, no, focal, consolidation, ...",[normal],40,1
1,CXR2277_IM-0864-1001;CXR2277_IM-0864-2001,CXR2277_IM-0864-1001,CXR2277_IM-0864-2001,None Available.,XXXX-year-old female with chest pain.,the lungs are clear without evidence of focal ...,no radiographic evidence of acute cardiopulmon...,normal,normal,the lungs are clear without evidence of focal ...,['normal'],normal,"[the, lungs, are, clear, without, evidence, of...",[normal],61,1
2,CXR1015_IM-0001-1001;CXR1015_IM-0001-2001;CXR1...,CXR1015_IM-0001-1001;CXR1015_IM-0013-1001,CXR1015_IM-0001-2001;CXR1015_IM-0013-2001,XXXX,"XXXX-year-old female, COPD exacerbation, short...",streaky and patchy bibasilar opacities triangu...,bibasilar opacities right greater than left fe...,Opacity/lung/base/bilateral/patchy/streaky;Tec...,Opacity;Technical Quality of Image Unsatisfact...,streaky and patchy bibasilar opacities triangu...,"['opacity', 'technical quality of image unsati...",opacity technical quality of image unsatisfactory,"[streak, ##y, and, patch, ##y, bi, ##bas, ##il...","[op, ##ac, ##ity, technical, quality, of, imag...",77,11
3,CXR1992_IM-0649-1001;CXR1992_IM-0649-4004,CXR1992_IM-0649-4004,CXR1992_IM-0649-1001,"XXXX, XXXX",Seizure,borderline heart size the lungs are hyperexpan...,findings of chronic obstructive pulmonary disease,Cardiomegaly/borderline;Lung/hyperdistention;L...,"Cardiomegaly;Lung;Lung, Hyperlucent;Pulmonary ...",borderline heart size the lungs are hyperexpan...,"['cardiomegaly', 'lung', 'lung hyperlucent', '...",cardiomegaly lung lung hyperlucent pulmonary d...,"[border, ##line, heart, size, the, lungs, are,...","[card, ##iom, ##ega, ##ly, lung, lung, hyper, ...",84,22
4,CXR979_IM-2466-1001;CXR979_IM-2466-2001,CXR979_IM-2466-1001,CXR979_IM-2466-2001,None.,XXXX-year-old female. Chest pain. Prior lumpec...,the cardiomediastinal silhouette is normal in ...,negative for acute abnormality,Lung/hyperdistention;Surgical Instruments/thor...,Lung;Surgical Instruments,the cardiomediastinal silhouette is normal in ...,"['lung', 'surgical instruments']",lung surgical instruments,"[the, card, ##iom, ##ed, ##ias, ##tina, ##l, s...","[lung, surgical, instruments]",59,3


In [15]:
train_df['Image_path'] = train_img_ids
train_df['Image_path'] = train_df['Image_path'].apply(lambda x : os.path.join(DATA_DIR,x+'.png'))

val_df['Image_path'] = val_img_ids
val_df['Image_path'] = val_df['Image_path'].apply(lambda x : os.path.join(DATA_DIR,x+'.png'))

test_df['Image_path'] = test_img_ids
test_df['Image_path'] = test_df['Image_path'].apply(lambda x : os.path.join(DATA_DIR,x+'.png'))

In [16]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(2566, 17)
(366, 17)
(733, 17)


In [17]:
def read_image(img_path,transform):
    img = skimage.io.imread(img_path)
    img = xrv.datasets.normalize(img, 255) 

    # Check that images are 2D arrays
    if len(img.shape) > 2:
        img = img[:, :, 0]
    if len(img.shape) < 2:
        print("error, dimension lower than 2 for image")

    # Add color channel
    img = img[None, :, :]
    img = transform(img)
    return img
    



class ImageCaptionDataset(Dataset):
    def __init__(self,image_paths,targets,keywords,imgsize,tfm,
                 encoder_max_length=config['encoder_max_length'],decoder_max_length=config['decoder_max_length']):
        self.image_paths = image_paths
        self.targets = targets
        self.keywords = keywords
        self.tfm = tfm
        self.size = imgsize
        self.encoder_max_length = encoder_max_length
        self.decoder_max_length = decoder_max_length
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#         self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        target = self.targets[idx]
        keywords = self.keywords[idx]
#         img = cv2.imread(img_path)
#         img = Image.open(img_path)

        img = read_image(img_path,self.tfm)
#         if self.tfm:
#             img = self.tfm(img)
        enc = self.tokenizer(keywords,add_special_tokens=True,padding='max_length',truncation=True,
                                 max_length=self.encoder_max_length,return_tensors='pt')
        enc_pos_ids = torch.ones(enc.input_ids.shape, dtype=torch.long)
        labels = self.tokenizer(target,add_special_tokens=True,padding='max_length',truncation=True,
                                 max_length=self.decoder_max_length).input_ids
        labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]
        labels = labels[1:] + [-100]
        return torch.tensor(img,dtype=torch.float32),enc.input_ids,enc.token_type_ids,enc.attention_mask,enc_pos_ids,torch.tensor(labels, dtype = torch.long)

In [18]:
configuration = VisualBertConfig()


In [19]:
configuration


VisualBertConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "bypass_transformer": false,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "visual_bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "special_visual_initialize": true,
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "visual_embedding_dim": 512,
  "vocab_size": 30522
}

In [20]:
# class Transfer(nn.Module):
#     def __init__(self,config):
#         super(Transfer, self).__init__()
#         self.model = models.resnet152(pretrained=True)
#         for p in self.model.parameters():   # originally it was self.parameters()
#             p.requires_grad=False
#         self.relu = nn.ReLU()
#         self.conv2 = nn.Conv2d(2048, config['hidden_size'], kernel_size=(1, 1), stride=(1, 1), bias=False)
#         self.gap2 = nn.AdaptiveAvgPool2d((1,1))
#         self.conv3 = nn.Conv2d(1024, config['hidden_size'], kernel_size=(1, 1), stride=(1, 1), bias=False)
#         self.gap3 = nn.AdaptiveAvgPool2d((1,1))
#         self.conv4 = nn.Conv2d(512, config['hidden_size'], kernel_size=(1, 1), stride=(1, 1), bias=False)
#         self.gap4 = nn.AdaptiveAvgPool2d((1,1))
#         self.conv5 = nn.Conv2d(256, config['hidden_size'], kernel_size=(1, 1), stride=(1, 1), bias=False)
#         self.gap5 = nn.AdaptiveAvgPool2d((1,1))
#         self.conv7 = nn.Conv2d(64, config['hidden_size'], kernel_size=(1, 1), stride=(1, 1), bias=False)
#         self.gap7 = nn.AdaptiveAvgPool2d((1,1))
#     def forward(self, img):
#         modules2 = list(self.model.children())[:-2]
#         fix2 = nn.Sequential(*modules2)
#         v_2 = self.gap2(self.relu(self.conv2(fix2(img)))).view(-1,config['hidden_size'])
#         modules3 = list(self.model.children())[:-3]
#         fix3 = nn.Sequential(*modules3)
#         v_3 = self.gap3(self.relu(self.conv3(fix3(img)))).view(-1,config['hidden_size'])
#         modules4 = list(self.model.children())[:-4]
#         fix4 = nn.Sequential(*modules4)
#         v_4 = self.gap4(self.relu(self.conv4(fix4(img)))).view(-1,config['hidden_size'])
#         modules5 = list(self.model.children())[:-5]
#         fix5 = nn.Sequential(*modules5)
#         v_5 = self.gap5(self.relu(self.conv5(fix5(img)))).view(-1,config['hidden_size'])
#         modules7 = list(self.model.children())[:-7]
#         fix7 = nn.Sequential(*modules7)
#         v_7 = self.gap7(self.relu(self.conv7(fix7(img)))).view(-1,config['hidden_size'])
#         return v_2, v_3, v_4, v_5, v_7
    
# class Embeddings(nn.Module):
#     def __init__(self, config):
#         super(Embeddings,self).__init__()
#         base_model = BertModel.from_pretrained('bert-base-uncased')
#         bert_model = nn.Sequential(*list(base_model.children())[0:])
#         self.bert_embedding = bert_model[0]
#         for p in self.bert_embedding.parameters():
#             p.requires_grad=False
#         self.trans = Transfer(config)
#     def forward(self, img, input_ids, token_type_ids, mask):
#         v_2, v_3, v_4, v_5, v_7 = self.trans(img)
#         h = self.bert_embedding(input_ids=input_ids, token_type_ids=token_type_ids,position_ids=None)
#         for i in range(len(h)):
#             h[i][1] = v_2[i]
#         for i in range(len(h)):
#             h[i][2] = v_3[i]
#         for i in range(len(h)):
#             h[i][3] = v_4[i]
#         for i in range(len(h)):
#             h[i][4] = v_5[i]
#         for i in range(len(h)):
#             h[i][5] = v_7[i] 
#         return h



class Transfer(nn.Module):
    def __init__(self,hidden_size=config['hidden_size']):
        super(Transfer, self).__init__()
        self.hidden_size = hidden_size
#         self.model = models.resnet152(pretrained=True)
        self.model = xrv.models.DenseNet(weights="densenet121-res224-all")
#         for p in self.model.parameters():
#             p.requires_grad=False
        self.relu = nn.ReLU()
        self.conv = nn.Conv2d(1024,self.hidden_size , kernel_size=(1, 1), stride=(1, 1))
    def forward(self, img):
        n = img.shape[0]
        modules2 = list(self.model.children())[:-2]
        fix2 = nn.Sequential(*modules2)
#         out = fix2(img)
        out = self.relu(self.conv(fix2(img)))
        out = out.view(n,self.hidden_size,-1)
        out = out.permute(0,2,1)
        return out

class Encoder(nn.Module):
    def __init__(self, config,device='cpu'):
        super(Encoder,self).__init__()
        
#         configuration = VisualBertConfig()
#         configuration.hidden_size = 512
#         self.visual_bert = VisualBertModel(configuration)
        
        self.visual_bert = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre")
#         self.bert_model = BertModel.from_pretrained('bert-base-uncased',
#                                   output_hidden_states = True, # Whether the model returns all hidden-states.
#                                   )
#         for p in self.bert_model.parameters():
#             p.requires_grad=False
        self.trans = Transfer(config['hidden_size'])
        self.layer_norm = nn.LayerNorm(config['hidden_size'])
        self.device = device
    def forward(self,img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids):
#         outputs = self.bert_model(input_ids=inp_ids,attention_mask=attn_mask)
#         hidden_states = outputs[2]
#         text_embed = hidden_states[0]
        visual_embeds = self.trans(img)
        visual_token_type_ids = torch.ones(visual_embeds.shape[:-1], dtype=torch.long).to(self.device)
        visual_attention_mask = torch.ones(visual_embeds.shape[:-1], dtype=torch.float).to(self.device)
        outputs = self.visual_bert(input_ids=enc_inp_ids,attention_mask=enc_attn_mask,token_type_ids=enc_seg_ids,
                                  position_ids=enc_pos_ids,visual_embeds=visual_embeds,
                                   visual_attention_mask=visual_attention_mask,visual_token_type_ids=visual_token_type_ids)
#         last_hidden_state = outputs.last_hidden_state
        return outputs

In [21]:
class ImageCaptionModel(nn.Module):
    def __init__(self,config,device):
        super(ImageCaptionModel,self).__init__()
        
        self.encoder = Encoder(config,device)
        self.encoder_decoder_model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", 
                                                            "bert-base-uncased")
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        self.encoder_decoder_model.config.decoder_start_token_id = self.tokenizer.cls_token_id
        self.encoder_decoder_model.config.eos_token_id = self.tokenizer.sep_token_id
        self.encoder_decoder_model.config.pad_token_id = self.tokenizer.pad_token_id
        self.encoder_decoder_model.config.vocab_size = self.encoder_decoder_model.config.encoder.vocab_size
        
    def forward(self,img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels):
        enc_outputs = self.encoder(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids)
        outputs = self.encoder_decoder_model(encoder_outputs=enc_outputs,
                                            labels=labels)
        loss, logits = outputs.loss, outputs.logits
        return logits,loss
    
    
    def generate_top_p(self,img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,top_k_params):
        enc_outputs = self.encoder(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids)
        sample_outputs = self.encoder_decoder_model.generate(encoder_outputs=enc_outputs,
                                                            do_sample=top_k_params['do_sample'], 
                                                            top_k=top_k_params['top_k'], 
                                                            top_p=top_k_params['top_p'], 
                                                            num_return_sequences=top_k_params['num_return_sequences'])
        return self.tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
            
    def generate_beam(self,img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,beam_params):
        enc_outputs = self.encoder(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids)
        sample_outputs = self.encoder_decoder_model.generate(encoder_outputs=enc_outputs,
                                                             max_length=beam_params['max_length'],
                                                             num_beams=beam_params['num_beams'],
                                                             no_repeat_ngram_size=beam_params['no_repeat_ngram_size'],
                                                             num_return_sequences=beam_params['num_return_sequences'],
                                                             early_stopping=beam_params['early_stopping']) 
        return self.tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)

In [22]:
tfm = transforms.Compose([transforms.Resize((224,224)),
                            transforms.ToTensor()])


transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])

dataset = ImageCaptionDataset(train_df['Image_path'].values,train_df['Target'].values,
                              train_df['Prepoc_Problems'].values,config['image_size'],transform)

img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels = dataset.__getitem__(0)
img = img.unsqueeze(0)
labels = labels.unsqueeze(0)

Setting XRayResizer engine to cv2 could increase performance.


In [23]:
beam_params = dict(
    min_length = 10,
    max_length = 280,
    num_beams = 6,
    no_repeat_ngram_size=2,
    num_return_sequences=1,
    early_stopping=True,
    length_penalty = 2.0
)


top_k_params = dict(
    min_length = 10,
    max_length = 280,
    do_sample = True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

ictModel = ImageCaptionModel(config,device).to(device)
optimizer = torch.optim.Adam(ictModel.parameters(), lr = 0.00001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.8, patience=2, verbose = True)
criterion = torch.nn.CrossEntropyLoss(reduction='none')

Downloading:   0%|          | 0.00/631 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/428M [00:00<?, ?B/s]

Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


If this fails you can run `wget https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt -O /root/.torchxrayvision/models_data/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt`
[██████████████████████████████████████████████████]


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relatio

In [25]:
def train_one_epoch(loader):

    ictModel.train()
    total_epoch_train_loss = 0
    total_epoch_valid_loss = 0
    total_train_words = 0
    total_valid_words = 0
    train_loss = []
    bar = tqdm(loader, leave = False)
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    for (img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels) in bar:
        img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels = img.to(device),enc_inp_ids.to(device),enc_seg_ids.to(device),enc_attn_mask.to(device),enc_pos_ids.to(device),labels.to(device)
        enc_inp_ids = enc_inp_ids.squeeze(1)
        enc_seg_ids = enc_seg_ids.squeeze(1)
        enc_attn_mask = enc_attn_mask.squeeze(1)
        enc_pos_ids = enc_pos_ids.squeeze(1)
        labels = labels.squeeze(1)
        
        
        loss_func = criterion
        optimizer.zero_grad()
        logits,loss = ictModel(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels)
        loss.backward()
        optimizer.step()
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        bar.set_description('train_loss: %.5f' % (loss_np))

    return np.mean(train_loss)

def validate(loader):
    ictModel.eval()
    val_loss = []

    beam_search_outputs = []
    top_p_outputs = []
    bar = tqdm(loader, leave=False)

    with torch.no_grad():
        for (img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels) in bar:
            img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels = img.to(device),enc_inp_ids.to(device),enc_seg_ids.to(device),enc_attn_mask.to(device),enc_pos_ids.to(device),labels.to(device)
            enc_inp_ids = enc_inp_ids.squeeze(1)
            enc_seg_ids = enc_seg_ids.squeeze(1)
            enc_attn_mask = enc_attn_mask.squeeze(1)
            enc_pos_ids = enc_pos_ids.squeeze(1)
            labels = labels.squeeze(1)
            
            logits,loss = ictModel(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels)
            loss_np = loss.detach().cpu().numpy()
            val_loss.append(loss_np)
            bar.set_description('val_loss: %.5f' % (loss_np))

        val_loss = np.mean(val_loss)
    return val_loss

def generate_captions(loader,model,device,params,beamSearch=True):
    model.eval()
    generated_outputs = []
    bar = tqdm(loader, leave=False)

    with torch.no_grad():
        for (img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels) in bar:
            img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,labels = img.to(device),enc_inp_ids.to(device),enc_seg_ids.to(device),enc_attn_mask.to(device),enc_pos_ids.to(device),labels.to(device)
            enc_inp_ids = enc_inp_ids.squeeze(1)
            enc_seg_ids = enc_seg_ids.squeeze(1)
            enc_attn_mask = enc_attn_mask.squeeze(1)
            enc_pos_ids = enc_pos_ids.squeeze(1)
            labels = labels.squeeze(1)
            
            if(beamSearch):
                sample_outputs = model.generate_beam(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,params)
            else:
                sample_outputs = model.generate_top_p(img,enc_inp_ids,enc_seg_ids,enc_attn_mask,enc_pos_ids,params)
            generated_outputs.append(sample_outputs)
    return generated_outputs



def calculate_rogue_score(sentence_list_1,sentence_list_2):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rogue1_scores = []
    rogueL_scores = []
    for sent1,sent2 in zip(sentence_list_1,sentence_list_2):
        scores = scorer.score(sent1,sent2)
        
        rogue1_scores.append(scores['rouge1'][2])
        rogueL_scores.append(scores['rougeL'][2])
        
    return np.mean(rogueL_scores),np.mean(rogue1_scores)


def calculate_meteor(pred_caps,act_caps):
    meteor_scores = []
    for pred,act in zip(pred_caps,act_caps):
        ref = act.split()
        hyp = pred.split()
        meteor_scores.append(meteor_score.meteor_score([ref],hyp))
    return np.mean(meteor_scores)


def calculate_bleu(pred_caps,act_caps,weights=(1, 0, 0, 0)):
    bleu_scores = []
    smoothie = SmoothingFunction().method4

    for pred,act in zip(pred_caps,act_caps):
        ref = act.split()
        hyp = pred.split()
        bleu_scores.append(bleu([ref], hyp, weights=weights))
    return np.mean(bleu_scores)

In [26]:
# train_tfm = transforms.Compose([transforms.Resize((224,224)),
#                             transforms.ToTensor()])

# val_tfm = transforms.Compose([transforms.Resize((224,224)),
#                             transforms.ToTensor()])

# test_tfm = transforms.Compose([transforms.Resize((224,224)),
#                             transforms.ToTensor()])



# train_tfm = transforms.Compose([transforms.Resize((224,224)), 
#                             transforms.RandomResizedCrop(224,scale=(0.95,1.05),ratio=(0.95,1.05)),
#                             transforms.RandomRotation(5),
#                             transforms.ColorJitter(brightness=0.05,contrast=0.05,saturation=0.05,hue=0.05),
#                             transforms.ToTensor(), 
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# val_tfm = transforms.Compose([transforms.Resize((224,224)), 
#                             transforms.RandomResizedCrop(224,scale=(0.95,1.05),ratio=(0.95,1.05)),
#                             transforms.RandomRotation(5),
#                             transforms.ColorJitter(brightness=0.05,contrast=0.05,saturation=0.05,hue=0.05),
#                             transforms.ToTensor(), 
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# test_tfm = transforms.Compose([transforms.Resize((224,224)), 
#                             transforms.RandomResizedCrop(224,scale=(0.95,1.05),ratio=(0.95,1.05)),
#                             transforms.RandomRotation(5),
#                             transforms.ColorJitter(brightness=0.05,contrast=0.05,saturation=0.05,hue=0.05),
#                             transforms.ToTensor(), 
#                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_tfm = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])

val_tfm = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])

test_tfm = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                xrv.datasets.XRayResizer(224)])

Setting XRayResizer engine to cv2 could increase performance.
Setting XRayResizer engine to cv2 could increase performance.
Setting XRayResizer engine to cv2 could increase performance.


In [27]:
traindataset =ImageCaptionDataset(train_df['Image_path'].values,train_df['Target'].values,
                              train_df['Prepoc_Problems'].values,config['image_size'],train_tfm)

valdataset =ImageCaptionDataset(val_df['Image_path'].values,val_df['Target'].values,
                              val_df['Prepoc_Problems'].values,config['image_size'],val_tfm)

testdataset =ImageCaptionDataset(test_df['Image_path'].values,test_df['Target'].values,
                              test_df['Prepoc_Problems'].values,config['image_size'],test_tfm)

trainloader = DataLoader(traindataset, batch_size = config['bs'], shuffle=True, num_workers = 2)
valloader = DataLoader(valdataset, batch_size = config['bs'], shuffle=False, num_workers = 2)
testloader = DataLoader(testdataset, batch_size = config['bs'], shuffle=False, num_workers = 2)

In [28]:
best_val_loss = np.inf
best_rogue_val = 0.0
val_target = list(val_df['Target'].values)
for epoch in range(config["epochs"]):
    
    print(f'Epoch {epoch+1}/{config["epochs"]}')

    train_loss = train_one_epoch(trainloader)
    val_loss = validate(valloader)
    
    search_outputs_val = generate_captions(valloader,ictModel,device,beam_params)
    val_preds = [out for cap in search_outputs_val for out in cap]
    val_rogue,_ = calculate_rogue_score(val_preds,val_target)
    
    scheduler.step(val_loss)
    
    
    content = f'Learning rate: {(optimizer.param_groups[0]["lr"]):.7f}, Train loss: {(train_loss):.4f},Val loss: {(val_loss):.4f},Val Rogue: {(val_rogue):.4f}'
    print(content)

    if val_loss < best_val_loss:
        torch.save(ictModel, './BestValModelLoss')
        best_val_loss = val_loss 
        
    if val_rogue > best_rogue_val:
        torch.save(ictModel, './BestValModelRogue')
        best_rogue_val = val_rogue 

Epoch 1/20


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

Learning rate: 0.0000100, Train loss: 3.8251,Val loss: 2.1915,Val Rogue: 0.2836
Epoch 2/20


...

Epoch    19: reducing learning rate of group 0 to 5.1200e-06.
Learning rate: 0.0000051, Train loss: 0.5218,Val loss: 1.3422,Val Rogue: 0.3188
Epoch 20/20


  0%|          | 0/642 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/92 [00:00<?, ?it/s]

Learning rate: 0.0000051, Train loss: 0.4863,Val loss: 1.3594,Val Rogue: 0.3217


In [29]:
search_outputs = generate_captions(testloader,ictModel,device,beam_params)

  0%|          | 0/184 [00:00<?, ?it/s]

In [30]:
test_preds = [out for cap in search_outputs for out in cap]
test_target = list(test_df['Target'].values)

In [31]:
with open("./hyp_beam.txt", 'w') as output:
    for cap in test_preds:
        output.write(str(cap) + '\n')
        
with open("./ref_beam.txt", 'w') as output:
    for cap in test_target:
        output.write(str(cap) + '\n')

In [32]:
rogueL_score_test,rogue1_score_test = calculate_rogue_score(test_preds,test_target)

print(rogueL_score_test)
print(rogue1_score_test)

0.3337440582093563
0.4317368915948421


In [33]:
ictModel = ImageCaptionModel(config,device)
ictModel = torch.load('/kaggle/working/BestValModelLoss')
ictModel = ictModel.to(device)
ictModel.eval()

Some weights of the model checkpoint at uclanlp/visualbert-vqa-coco-pre were not used when initializing VisualBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing VisualBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing B

ImageCaptionModel(
  (encoder): Encoder(
    (visual_bert): VisualBertModel(
      (embeddings): VisualBertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=1)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (visual_token_type_embeddings): Embedding(2, 768)
        (visual_position_embeddings): Embedding(512, 768)
        (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
      )
      (encoder): VisualBertEncoder(
        (layer): ModuleList(
          (0): VisualBertLayer(
            (attention): VisualBertAttention(
              (self): VisualBertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_featu

In [34]:
search_outputs = generate_captions(testloader,ictModel,device,beam_params)
test_preds = [out for cap in search_outputs for out in cap]

  0%|          | 0/184 [00:00<?, ?it/s]

In [35]:
rogueL_score_test,rogue1_score_test = calculate_rogue_score(test_preds,test_target)

print(rogueL_score_test)
print(rogue1_score_test)

0.31586525719814196
0.4025590498643963


In [36]:
test_preds[:10]

['the heart is normal in size the mediastinum is unremarkable the lungs are clear no acute disease',
 'the heart is normal in size the mediastinum is unremarkable the lungs are clear no acute disease',
 'heart size within normal limits stable mediastinal and hilar contours no focal alveolar consolidation no definite pleural effusion seen bronchovascular crowding without typical findings of pulmonary edema',
 'the heart is normal in size the mediastinum is stable the lungs are hyperinflated but clear there is no pleural effusion or pneumothorax the osseous structures are intact chronic lung disease without acute disease',
 'the cardiomediastinal silhouette and pulmonary vasculature are within normal limits in size the lungs are clear of focal airspace disease pneumothorax or pleural effusion there are no acute bony findings the surgical clips are intact negative for acute cardiopulmonary findings',
 'the heart is normal in size the mediastinum is unremarkable the lungs are clear no acut